Si vogliono estrarre alcune informazioni dai dataset riguardanti la regione Lombardia. In particolare si vogliono avere informazioni su:

1. L'elenco dei comuni con i fiumi che li attraversano (nome comune e nome fiume).

2. Contare per ogni comune il numero di fiumi che li attraversa. Se un comune non è attraversato da un fiume, indicare 0.

3. L'elenco delle province con le autostrade che le attraversano. Visualizzare anche le province che non sono attraversate da autostrade.

4. Visualizzare su una mappa i risultati dell'esercizio 3, facendo in modo che le province che non sono attraversate da autostrade siano solo contornate di nero, mentre le altre province devono essere colorate a seconda del numero di autostrade che le attraversano.

5. Visualizzare su una mappa i quartieri di Milano colorati a seconda del numero di fontanelle presenti.

6. Visualizzare l'elenco dei quartieri di Milano che confinano con comuni fuori Milano. Indicare il nome del quartiere e il nome del comune con cui confina.

In [ ]:
import os
!pip install rtree
!curl -L http://download.osgeo.org/libspatialindex/spatialindex-src-1.8.5.tar.gz | tar xz
os.chdir("/content/spatialindex-src-1.8.5")
!./configure
!make
!make install
!ldconfig

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install geopandas
!pip install contextily

In [ ]:
%matplotlib inline

import pandas as pd
import geopandas
import contextily

In [ ]:
comuni = geopandas.read_file('/content/drive/MyDrive/Colab Notebooks/Geopandas - Materiale esercizi/Limiti01012021_g/Com01012021_g')
fiumi = geopandas.read_file('/content/drive/MyDrive/Colab Notebooks/Geopandas - Materiale esercizi/Corsi_acqua_Piano_di_Gestione.zip')
autostrade = geopandas.read_file('/content/drive/MyDrive/Colab Notebooks/Geopandas - Materiale esercizi/Autostrade_10000_CT10_line.zip')
quartieri_milano = geopandas.read_file('/content/drive/MyDrive/Colab Notebooks/Geopandas - Materiale esercizi/ds964_nil_wm.zip')
fontanelle = geopandas.read_file('/content/drive/MyDrive/Colab Notebooks/Geopandas - Materiale esercizi/Fontanelle.zip')
province = geopandas.read_file('/content/drive/MyDrive/Colab Notebooks/Geopandas - Materiale esercizi/Limiti01012021_g/ProvCM01012021_g')
regioni = geopandas.read_file('/content/drive/MyDrive/Colab Notebooks/Geopandas - Materiale esercizi/Limiti01012021_g/Reg01012021_g')

1. Visualizzare l'elenco dei comuni con i fiumi che li attraversano (nome comune e nome fiume).

In [ ]:
fiumi_lomb = fiumi[fiumi['REGIONE']=='Lombardia']
fiumi_lomb

In [ ]:
lombardia = regioni[regioni['DEN_REG']=='Lombardia']
lombardia

,COD_RIP,COD_REG,DEN_REG,Shape_Leng,Shape_Area,geometry
2,1,3,Lombardia,1.410223e+06,2.386232e+10,"MULTIPOLYGON (((485536.366 4985444.605, 485366..."


In [82]:
milano = province[province['DEN_UTS']=='Milano']
milano

,COD_RIP,COD_REG,COD_PROV,COD_CM,COD_UTS,DEN_PROV,DEN_CM,DEN_UTS,SIGLA,TIPO_UTS,Shape_Leng,Shape_Area,geometry
14,1,3,15,215,215,-,Milano,Milano,MI,Citta metropolitana,309951.219188,1.575033e+09,"MULTIPOLYGON (((538136.289 5004372.814, 539162..."


In [ ]:
comuni_lomb = comuni[comuni.within(lombardia.geometry.squeeze())]
comuni_lomb

,COD_RIP,COD_REG,COD_PROV,COD_CM,COD_UTS,PRO_COM,PRO_COM_T,COMUNE,COMUNE_A,CC_UTS,Shape_Leng,Shape_Area,geometry
1448,1,3,12,0,12,12001,012001,Agra,None,0,6996.108320,2.942389e+06,"POLYGON ((482802.662 5097733.142, 482784.662 5..."
1449,1,3,12,0,12,12002,012002,Albizzate,None,0,9255.748026,3.813831e+06,"POLYGON ((485235.221 5064709.066, 485545.361 5..."
1450,1,3,12,0,12,12003,012003,Angera,None,0,24194.847570,1.748219e+07,"POLYGON ((470218.756 5071198.335, 470937.656 5..."
1451,1,3,12,0,12,12004,012004,Arcisate,None,0,19072.979574,1.229388e+07,"POLYGON ((488207.686 5080963.765, 488368.654 5..."
1452,1,3,12,0,12,12005,012005,Arsago Seprio,None,0,15318.402658,1.053346e+07,"POLYGON ((478637.631 5063107.396, 478750.600 5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7741,1,3,108,0,108,108051,108051,Busnago,None,0,11376.029586,5.834750e+06,"POLYGON ((535742.586 5052473.474, 535839.585 5..."
7742,1,3,108,0,108,108052,108052,Caponago,None,0,11874.711735,4.807794e+06,"POLYGON ((530927.622 5047008.514, 530890.358 5..."
7743,1,3,108,0,108,108053,108053,Cornate d'Adda,None,0,15414.910444,1.364665e+07,"POLYGON ((537678.573 5057235.438, 537951.570 5..."
7744,1,3,108,0,108,108054,108054,Lentate sul Seveso,None,0,17060.616570,1.431746e+07,"POLYGON ((508523.788 5059797.421, 509626.779 5..."


In [ ]:
join_comuni_fiumi = geopandas.sjoin(fiumi, comuni_lomb, op='intersects', how='left')
join_comuni_fiumi

In [37]:
comuni_fiumi = join_comuni_fiumi[['NOME', 'COMUNE']].sort_values(by='COMUNE')
comuni_fiumi

,NOME,COMUNE
1,Scolmatore Piene Nord - Ovest (Canale),Abbiategrasso
0,Naviglio Grande (Canale),Abbiategrasso
7,Ticino (Fiume),Abbiategrasso
18,Villoresi (Canale),Agrate Brianza
1,Scolmatore Piene Nord - Ovest (Canale),Albairate
...,...,...
30,Ticino (Fiume),Vigevano
31,Naviglio Martesana (Canale),Vimodrone
13,Lambro (Fiume),Vizzolo Predabissi
5,Addetta (Colatore),Vizzolo Predabissi


2. Contare per ogni comune il numero di fiumi che li attraversa. Se un comune non è attraversato da un fiume, indicare 0.

3. Visualizzare l'elenco delle province con le autostrade che le attraversano. Visualizzare anche le province che non sono attraversate da autostrade.

In [52]:
autostrade_lomb = autostrade[autostrade.within(lombardia.geometry.squeeze())]

In [53]:
province_lomb = province[province.within(lombardia.geometry.squeeze())]

In [67]:
prov_autostrade = geopandas.sjoin(autostrade_lomb, province_lomb, how='inner')
prov_autostrade

,TS_EID,COD_PE,PERCORSO,geometry,index_right,COD_RIP,COD_REG,COD_PROV,COD_CM,COD_UTS,DEN_PROV,DEN_CM,DEN_UTS,SIGLA,TIPO_UTS,Shape_Leng,Shape_Area
3,65.0,A50,TANGENZIALE OVEST DI MILANO,"LINESTRING (521699.514 5024872.823, 521630.139...",14,1,3,15,215,215,-,Milano,Milano,MI,Citta metropolitana,309951.219188,1.575033e+09
4,62.0,A50,TANGENZIALE OVEST DI MILANO,"LINESTRING (504384.530 5044095.266, 504414.405...",14,1,3,15,215,215,-,Milano,Milano,MI,Citta metropolitana,309951.219188,1.575033e+09
5,90.0,A51,TANGENZIALE EST DI MILANO,"LINESTRING (519364.775 5030574.241, 519401.151...",14,1,3,15,215,215,-,Milano,Milano,MI,Citta metropolitana,309951.219188,1.575033e+09
6,78.0,A51,TANGENZIALE EST DI MILANO,"LINESTRING (519346.650 5030596.741, 519377.151...",14,1,3,15,215,215,-,Milano,Milano,MI,Citta metropolitana,309951.219188,1.575033e+09
10,130.0,A8,AUTOSTRADA MILANO- VARESE,"LINESTRING (510188.551 5038184.696, 510192.926...",14,1,3,15,215,215,-,Milano,Milano,MI,Citta metropolitana,309951.219188,1.575033e+09
11,131.0,A8,AUTOSTRADA MILANO- VARESE,"LINESTRING (510188.551 5038184.696, 510186.301...",14,1,3,15,215,215,-,Milano,Milano,MI,Citta metropolitana,309951.219188,1.575033e+09
15,140.0,A9,AUTOSTRADA LAINATE-COMO-CHIASSO,"LINESTRING (500870.596 5047376.792, 500867.222...",14,1,3,15,215,215,-,Milano,Milano,MI,Citta metropolitana,309951.219188,1.575033e+09
16,1915.0,A4,AUTOSTRADA TORINO-TRIESTE,"LINESTRING (509605.078 5040334.699, 509647.828...",14,1,3,15,215,215,-,Milano,Milano,MI,Citta metropolitana,309951.219188,1.575033e+09
17,1916.0,A4,AUTOSTRADA TORINO-TRIESTE,"LINESTRING (509605.078 5040334.699, 509629.203...",14,1,3,15,215,215,-,Milano,Milano,MI,Citta metropolitana,309951.219188,1.575033e+09
20,45.0,A4,AUTOSTRADA TORINO-TRIESTE,"LINESTRING (483693.626 5035220.147, 483734.501...",14,1,3,15,215,215,-,Milano,Milano,MI,Citta metropolitana,309951.219188,1.575033e+09


In [ ]:
autostrade_lomb['COD_PE']

In [69]:
prov_autostrade[['PERCORSO', 'DEN_UTS']]

,PERCORSO,DEN_UTS
3,TANGENZIALE OVEST DI MILANO,Milano
4,TANGENZIALE OVEST DI MILANO,Milano
5,TANGENZIALE EST DI MILANO,Milano
6,TANGENZIALE EST DI MILANO,Milano
10,AUTOSTRADA MILANO- VARESE,Milano
11,AUTOSTRADA MILANO- VARESE,Milano
15,AUTOSTRADA LAINATE-COMO-CHIASSO,Milano
16,AUTOSTRADA TORINO-TRIESTE,Milano
17,AUTOSTRADA TORINO-TRIESTE,Milano
20,AUTOSTRADA TORINO-TRIESTE,Milano


4. Visualizzare su una mappa i risultati dell'esercizio 3, facendo in modo che le province che non sono attraversate da autostrade siano solo contornate di nero, mentre le altre province devono essere colorate a seconda del numero di autostrade che le attraversano.

In [ ]:
province_attr = province_lomb[province_lomb.intersects(autostrade_lomb.geometry.squeeze())]

In [ ]:
ax = autostrade_lomb.to_crs(epsg=3857).plot(figsize=(18, 18))
province_attr.to_crs(epsg=3857).plot(ax=ax, facecolor='blue', alpha=0.5)
contextily.add_basemap(ax)

5. Visualizzare su una mappa i quartieri di Milano colorati a seconda del numero di fontanelle presenti.

In [38]:
geoFontQuart = geopandas.sjoin(fontanelle, quartieri_milano, op='within', how='left')
geoFontQuart

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if self.run_code(code, result):


,Fonte_left,Licenza,geometry,index_right,ID_NIL,NIL,Valido_dal,Valido_al,Fonte_right,Shape_Leng,Shape_Area,OBJECTID
0,© OpenStreetMap contributors - OSM,Licenza ODbL - http://www.openstreetmap.org/co...,POINT Z (9.16165 45.47091 0.00000),83.0,68.0,PAGANO,05/02/2020,Vigente,Milano 2030 - PGT Approvato,5221.836696,1.289686e+06,172.0
1,© OpenStreetMap contributors - OSM,Licenza ODbL - http://www.openstreetmap.org/co...,POINT Z (9.23184 45.46831 0.00000),76.0,22.0,CITTA' STUDI,05/02/2020,Vigente,Milano 2030 - PGT Approvato,6007.783217,2.207404e+06,165.0
2,© OpenStreetMap contributors - OSM,Licenza ODbL - http://www.openstreetmap.org/co...,POINT Z (9.14380 45.48017 0.00000),18.0,58.0,DE ANGELI - MONTE ROSA,05/02/2020,Vigente,Milano 2030 - PGT Approvato,5765.950197,1.323029e+06,107.0
3,© OpenStreetMap contributors - OSM,Licenza ODbL - http://www.openstreetmap.org/co...,POINT Z (9.14401 45.48319 0.00000),18.0,58.0,DE ANGELI - MONTE ROSA,05/02/2020,Vigente,Milano 2030 - PGT Approvato,5765.950197,1.323029e+06,107.0
4,© OpenStreetMap contributors - OSM,Licenza ODbL - http://www.openstreetmap.org/co...,POINT Z (9.11746 45.49320 0.00000),47.0,65.0,Q.RE GALLARATESE - Q.RE SAN LEONARDO - LAMPUGNANO,05/02/2020,Vigente,Milano 2030 - PGT Approvato,11053.767762,3.895679e+06,136.0
...,...,...,...,...,...,...,...,...,...,...,...,...
77,© OpenStreetMap contributors - OSM,Licenza ODbL - http://www.openstreetmap.org/co...,POINT Z (9.22943 45.47751 0.00000),76.0,22.0,CITTA' STUDI,05/02/2020,Vigente,Milano 2030 - PGT Approvato,6007.783217,2.207404e+06,165.0
78,© OpenStreetMap contributors - OSM,Licenza ODbL - http://www.openstreetmap.org/co...,POINT Z (9.22730 45.47965 0.00000),76.0,22.0,CITTA' STUDI,05/02/2020,Vigente,Milano 2030 - PGT Approvato,6007.783217,2.207404e+06,165.0
79,© OpenStreetMap contributors - OSM,Licenza ODbL - http://www.openstreetmap.org/co...,POINT Z (9.22773 45.47728 0.00000),76.0,22.0,CITTA' STUDI,05/02/2020,Vigente,Milano 2030 - PGT Approvato,6007.783217,2.207404e+06,165.0
80,© OpenStreetMap contributors - OSM,Licenza ODbL - http://www.openstreetmap.org/co...,POINT Z (9.22815 45.47994 0.00000),76.0,22.0,CITTA' STUDI,05/02/2020,Vigente,Milano 2030 - PGT Approvato,6007.783217,2.207404e+06,165.0


In [39]:
nFont = geoFontQuart.groupby('NIL').count()[['Shape_Leng']].reset_index().rename(columns={'Shape_Leng': 'Fontanelle'})
nFont

,NIL,Fontanelle
0,ADRIANO,1
1,BAGGIO - Q.RE DEGLI OLMI - Q.RE VALSESIA,2
2,BANDE NERE,1
3,BOVISA,1
4,BRUZZANO,1
5,BUENOS AIRES - PORTA VENEZIA - PORTA MONFORTE,1
6,CASCINA MERLATA,2
7,CIMIANO - ROTTOLE - Q.RE FELTRE,2
8,CITTA' STUDI,6
9,DE ANGELI - MONTE ROSA,2


In [ ]:
geoCountComuni = pd.merge(fontanelle, quartieri_milano, on='NIL', how='inner')
nFont.plot(column='Fontalle', legend=True, alpha=0.7)

6. Visualizzare l'elenco dei quartieri di Milano che confinano con comuni fuori Milano. Indicare il nome del quartiere e il nome del comune con cui confina.

In [ ]:
comuni_fuori = comuni[~comuni.within(milano.geometry.squeeze())]
comuni_fuori

In [ ]:
comuni_fuori.crs

In [122]:
quartieri_milano = quartieri_milano.to_crs(epsg=32632)

In [ ]:
quartieri_milano.crs

In [ ]:
quartieri_conf = quartieri_milano[quartieri_milano.touches(comuni_fuori.geometry.squeeze())]